<a href="https://colab.research.google.com/github/marcmontb/VC-sourcing-engine/blob/main/AI_scraping_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit pandas numpy requests matplotlib seaborn beautifulsoup4 duckduckgo_search pillow pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.3 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
# Paste the entire Streamlit app code here (from the artifact I provided)

import streamlit as st
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import time
import re
import os
import io
import json
from bs4 import BeautifulSoup
from PIL import Image
from datetime import datetime

# Set page configuration
st.set_page_config(
    page_title="CRM Data Enrichment Tool",
    page_icon="📊",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Application title and description
st.title("CRM Data Enrichment Tool")
st.subheader("Inspired by Coherent - Online data made easy")

# Sidebar
with st.sidebar:
    st.header("Settings")

    # API Key settings (optional)
    st.subheader("API Settings (Optional)")
    use_api_keys = st.checkbox("Use API Keys", value=False)

    if use_api_keys:
        openai_api_key = st.text_input("OpenAI API Key", type="password")
        serper_api_key = st.text_input("Serper API Key", type="password")

        if openai_api_key:
            os.environ["OPENAI_API_KEY"] = openai_api_key
        if serper_api_key:
            os.environ["SERPER_API_KEY"] = serper_api_key

    # Search settings
    st.subheader("Search Settings")
    search_delay = st.slider("Delay between searches (seconds)", 1, 10, 2)
    max_search_results = st.slider("Max search results per company", 1, 10, 3)

    # About section
    st.markdown("---")
    st.markdown("### About")
    st.markdown("This tool helps enrich your CRM data by researching companies online and extracting useful information.")
    st.markdown("Upload your CSV with company data, customize your settings, and get enhanced insights.")

# Main functions (adapted from the original script)
# Web Research Functions
@st.cache_data(ttl=3600)  # Cache results for 1 hour
def search_company(company_name, domain=None):
    """Search for company information online"""
    search_query = company_name
    if domain:
        search_query += f" {domain}"

    results = []

    # Try to import and use duckduckgo_search
    try:
        from duckduckgo_search import DDGS
        ddgs = DDGS()

        # First, try to get some results from DuckDuckGo
        search_results = ddgs.text(search_query, max_results=max_search_results)
        results = list(search_results)

        # If we have a domain, also search specifically on that domain
        if domain and domain.startswith(('http://', 'https://')):
            try:
                site_domain = domain.split('//')[1].split('/')[0]
                site_results = ddgs.text(f"site:{site_domain}", max_results=2)
                results.extend(list(site_results))
            except:
                pass
    except Exception as e:
        st.warning(f"Search error: {e}")
        # Fallback to a simple message if search fails
        results = [{"body": f"Information about {company_name}", "href": domain if domain else ""}]

    return results

@st.cache_data(ttl=3600)  # Cache results for 1 hour
def extract_website_info(url):
    """Extract text information from a website"""
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Remove scripts, styles, and other non-content tags
            for script in soup(["script", "style", "meta", "noscript", "header", "footer"]):
                script.extract()

            # Extract title
            title = soup.title.string if soup.title else ""

            # Extract meta description
            meta_desc = ""
            meta_tag = soup.find("meta", attrs={"name": "description"})
            if meta_tag and "content" in meta_tag.attrs:
                meta_desc = meta_tag["content"]

            # Extract main text content
            text = soup.get_text(separator=' ', strip=True)
            text = re.sub(r'\s+', ' ', text)  # Normalize whitespace

            # Extract potential "About" section
            about_section = ""
            about_headers = soup.find_all(["h1", "h2", "h3"], string=re.compile(r'about|company|who we are', re.I))
            for header in about_headers:
                section = []
                for sibling in header.find_next_siblings():
                    if sibling.name in ["h1", "h2", "h3"]:
                        break
                    section.append(sibling.get_text(strip=True))
                about_section += " ".join(section)

            return {
                "title": title,
                "meta_description": meta_desc,
                "about_section": about_section,
                "full_text": text[:5000],  # Limit to first 5000 chars
                "url": url
            }
        else:
            return {"error": f"Failed to fetch {url}: HTTP {response.status_code}"}
    except Exception as e:
        return {"error": f"Error processing {url}: {str(e)}"}

# Industry Classification
def classify_industry(company_text):
    """Classify company industry based on text description"""

    # Dictionary of industry keywords
    industry_keywords = {
        "TravelTech": ["travel", "tourism", "booking", "flight", "hotel", "vacation", "destination"],
        "HRTech": ["recruitment", "talent", "hiring", "hr ", "human resources", "workforce", "staffing"],
        "FinTech": ["finance", "banking", "insurance", "payment", "financial", "loan", "invest"],
        "HealthTech": ["health", "medical", "healthcare", "patient", "doctor", "hospital", "clinic"],
        "EdTech": ["education", "learning", "teaching", "school", "student", "academic", "course"],
        "ECommerce": ["ecommerce", "online shop", "online store", "shopping", "retail", "marketplace"],
        "SaaS": ["software as a service", "saas", "subscription software", "cloud service"],
        "Manufacturing": ["manufacturing", "factory", "production", "industrial", "equipment"],
        "IT Services": ["it services", "consulting", "system integration", "tech support"],
        "AgTech": ["agriculture", "farming", "crop", "livestock", "agri-tech"]
    }

    # Count matches for each industry
    matches = {}
    company_text = company_text.lower()

    for industry, keywords in industry_keywords.items():
        count = sum(1 for keyword in keywords if keyword.lower() in company_text)
        if count > 0:
            matches[industry] = count

    # Return the industry with the most keyword matches
    if matches:
        return max(matches.items(), key=lambda x: x[1])[0]
    else:
        return "Unknown"

# Helper functions for specific indicators
def has_sales_jobs(company_text):
    """Detect if company has sales job openings"""
    sales_keywords = [
        "sales representative", "sales manager", "sales executive", "sales job",
        "hiring sales", "sales position", "sales opportunity", "sales career"
    ]
    return any(keyword in company_text.lower() for keyword in sales_keywords)

def has_hr_team(company_text):
    """Detect if company has an HR team"""
    hr_keywords = [
        "hr team", "human resources team", "recruiting team", "talent team",
        "head of hr", "hr manager", "hr department", "people operations"
    ]
    return any(keyword in company_text.lower() for keyword in hr_keywords)

def has_uk_presence(company_text):
    """Detect if company has UK presence"""
    uk_keywords = [
        "uk office", "united kingdom", "london office", "manchester", "birmingham",
        "uk team", "uk based", "offices in the uk", "uk headquarters"
    ]
    return any(keyword in company_text.lower() for keyword in uk_keywords)

def has_ecommerce(company_text):
    """Detect if company has an ecommerce store"""
    ecommerce_keywords = [
        "shop now", "add to cart", "buy online", "online store", "ecommerce",
        "shopping cart", "checkout", "product page", "online shop"
    ]
    return any(keyword in company_text.lower() for keyword in ecommerce_keywords)

def mentions_ai(company_text):
    """Detect if company mentions AI"""
    ai_keywords = [
        "artificial intelligence", "machine learning", "deep learning", "ai ",
        "neural network", "natural language processing", "nlp", "computer vision"
    ]
    return any(keyword in company_text.lower() for keyword in ai_keywords)

# Main enrichment function
def enrich_company_data(df, progress_bar):
    """Enrich company data with web research"""
    enriched_df = df.copy()

    # Add enrichment columns if they don't exist
    new_columns = [
        'detailed_industry', 'has_sales_jobs', 'has_hr_team',
        'has_uk_presence', 'ecommerce_store', 'ai_mentions', 'data_confidence'
    ]

    for col in new_columns:
        if col not in enriched_df.columns:
            enriched_df[col] = None

    # Process each company
    for idx, row in enriched_df.iterrows():
        company_name = row['company_name']

        # Get domain from different possible column names
        domain = None
        for domain_col in ['company_domain', 'company_website', 'website', 'domain']:
            if domain_col in row and pd.notna(row[domain_col]):
                domain = row[domain_col]
                # Ensure domain has http/https prefix
                if domain and not domain.startswith(('http://', 'https://')):
                    domain = 'https://' + domain
                break

        # Update progress bar text and value
        progress_text = f"Processing {company_name}"
        progress_bar.progress((idx + 1) / len(enriched_df), text=progress_text)

        # Search for company information
        search_results = search_company(company_name, domain)

        # Aggregate text from search results
        all_text = ""
        confidence = 0

        # If we have a domain, try to extract info directly from company website
        website_info = {}
        if domain:
            website_info = extract_website_info(domain)
            if 'error' not in website_info:
                all_text += website_info.get('meta_description', '') + " "
                all_text += website_info.get('about_section', '') + " "
                all_text += website_info.get('full_text', '')
                confidence += 1

        # Process search results
        for i, result in enumerate(search_results):
            if isinstance(result, dict) and 'body' in result:
                all_text += result['body'] + " "
                confidence += 0.5

                # Try to extract more details from the first few results
                if i < 2 and 'href' in result:
                    site_info = extract_website_info(result['href'])
                    if 'error' not in site_info:
                        all_text += site_info.get('full_text', '')
                        confidence += 0.5

        # Perform analysis
        if all_text:
            # Classify industry
            detailed_industry = classify_industry(all_text)
            enriched_df.at[idx, 'detailed_industry'] = detailed_industry

            # Detect various indicators
            enriched_df.at[idx, 'has_sales_jobs'] = has_sales_jobs(all_text)
            enriched_df.at[idx, 'has_hr_team'] = has_hr_team(all_text)
            enriched_df.at[idx, 'has_uk_presence'] = has_uk_presence(all_text)
            enriched_df.at[idx, 'ecommerce_store'] = has_ecommerce(all_text)
            enriched_df.at[idx, 'ai_mentions'] = mentions_ai(all_text)

            # Set confidence score (0-5)
            enriched_df.at[idx, 'data_confidence'] = min(confidence, 5)
        else:
            enriched_df.at[idx, 'data_confidence'] = 0

        # Avoid rate limiting
        time.sleep(search_delay)

    # Complete the progress bar
    progress_bar.progress(1.0, text="Processing complete!")
    return enriched_df

# Visualization Functions
def create_visualizations(df):
    """Create various visualizations of the enriched data"""

    # Create columns for layout
    col1, col2 = st.columns(2)

    with col1:
        # Industry Distribution
        st.subheader("Industry Distribution")
        if 'detailed_industry' in df.columns:
            industry_counts = df['detailed_industry'].value_counts()
            fig, ax = plt.subplots(figsize=(10, 6))
            sns.barplot(x=industry_counts.index, y=industry_counts.values, ax=ax)
            plt.title('Company Distribution by Industry')
            plt.xlabel('Industry')
            plt.ylabel('Number of Companies')
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            st.pyplot(fig)
        else:
            st.info("Industry data not available.")

        # Data Confidence
        st.subheader("Data Confidence Distribution")
        if 'data_confidence' in df.columns:
            fig, ax = plt.subplots(figsize=(10, 6))
            sns.histplot(df['data_confidence'], bins=5, kde=True, ax=ax)
            plt.title('Data Confidence Distribution')
            plt.xlabel('Confidence Score (0-5)')
            plt.ylabel('Number of Companies')
            plt.tight_layout()
            st.pyplot(fig)
        else:
            st.info("Confidence data not available.")

    with col2:
        # Binary Indicators
        st.subheader("Company Indicators")
        indicators = ['has_sales_jobs', 'has_hr_team', 'has_uk_presence', 'ecommerce_store', 'ai_mentions']
        valid_indicators = [col for col in indicators if col in df.columns]

        if valid_indicators:
            counts = {}
            for indicator in valid_indicators:
                true_count = df[indicator].sum()
                false_count = len(df) - true_count
                counts[indicator] = [true_count, false_count]

            fig, ax = plt.subplots(figsize=(10, 8))
            x = np.arange(len(counts))
            width = 0.35

            # Plot
            ax.bar(x - width/2, [counts[ind][0] for ind in counts], width, label='Yes')
            ax.bar(x + width/2, [counts[ind][1] for ind in counts], width, label='No')

            ax.set_xlabel('Indicator')
            ax.set_ylabel('Number of Companies')
            ax.set_title('Company Indicators Distribution')
            ax.set_xticks(x)
            ax.set_xticklabels([ind.replace('_', ' ').replace('has ', '').title() for ind in counts.keys()])
            plt.xticks(rotation=45, ha='right')
            ax.legend()
            plt.tight_layout()
            st.pyplot(fig)
        else:
            st.info("Indicator data not available.")

        # Summary metrics
        st.subheader("Summary Metrics")
        metrics_col1, metrics_col2 = st.columns(2)

        # Calculate metrics
        total_companies = len(df)
        companies_with_sales = df['has_sales_jobs'].sum() if 'has_sales_jobs' in df.columns else 0
        companies_with_hr = df['has_hr_team'].sum() if 'has_hr_team' in df.columns else 0
        companies_with_ecommerce = df['ecommerce_store'].sum() if 'ecommerce_store' in df.columns else 0
        companies_with_ai = df['ai_mentions'].sum() if 'ai_mentions' in df.columns else 0

        with metrics_col1:
            st.metric("Total Companies", total_companies)
            st.metric("Companies with Sales Jobs", f"{companies_with_sales} ({int(companies_with_sales/total_companies*100 if total_companies else 0)}%)")
            st.metric("Companies with HR Team", f"{companies_with_hr} ({int(companies_with_hr/total_companies*100 if total_companies else 0)}%)")

        with metrics_col2:
            st.metric("Companies with Ecommerce", f"{companies_with_ecommerce} ({int(companies_with_ecommerce/total_companies*100 if total_companies else 0)}%)")
            st.metric("Companies mentioning AI", f"{companies_with_ai} ({int(companies_with_ai/total_companies*100 if total_companies else 0)}%)")

# Main application flow
def main():
    # Create tabs
    tab1, tab2, tab3 = st.tabs(["Upload & Process", "Results & Visualization", "Export"])

    # Tab 1: Upload and Process
    with tab1:
        st.header("Upload Company Data")

        # File uploader
        uploaded_file = st.file_uploader("Choose a CSV file with company data", type=['csv', 'xlsx'])

        if uploaded_file is not None:
            # Read the data
            try:
                if uploaded_file.name.endswith('.csv'):
                    df = pd.read_csv(uploaded_file)
                else:
                    df = pd.read_excel(uploaded_file)

                # Check required columns
                if 'company_name' not in df.columns:
                    st.error("The uploaded file must contain a 'company_name' column.")
                    st.stop()

                # Display original data
                st.subheader("Original Data")
                st.dataframe(df)

                # Store the dataframe in session state for access across tabs
                st.session_state.original_df = df

                # Process options
                st.subheader("Processing Options")

                # Allow limiting the number of companies to process
                process_all = st.checkbox("Process all companies", value=df.shape[0] <= 10)

                if not process_all:
                    max_companies = st.slider("Number of companies to process", 1, min(df.shape[0], 30), min(5, df.shape[0]))
                else:
                    max_companies = df.shape[0]

                # Custom indicators
                st.subheader("Custom Indicators (Coming Soon)")
                st.info("In a future version, you'll be able to define your own indicators here.")

                # Start processing button
                if st.button("Start Enrichment Process"):
                    # Create a subset of data if needed
                    process_df = df.head(max_companies).copy()

                    # Set up a progress bar
                    st.subheader("Processing Progress")
                    progress_bar = st.progress(0, text="Starting...")

                    # Process the data
                    with st.spinner('Searching the web for company information...'):
                        enriched_df = enrich_company_data(process_df, progress_bar)

                    # Store enriched data in session state
                    st.session_state.enriched_df = enriched_df

                    # Success message
                    st.success(f"✅ Successfully processed {len(enriched_df)} companies!")

                    # Prompt to view results
                    st.info("Click on the 'Results & Visualization' tab to view the enriched data.")

            except Exception as e:
                st.error(f"Error processing file: {e}")
                st.exception(e)

        else:
            # Show sample data option if no file uploaded
            if st.button("Use sample data"):
                # Create sample data
                sample_data = {
                    'company_name': ['TechNova Solutions', 'Green Leaf Organics', 'MediHealth Systems',
                                     'Global Travel Partners', 'EduLearn Academy'],
                    'company_domain': ['technova.com', 'greenleaforganics.co.uk', 'medihealthsystems.org',
                                      'globaltravelpartners.net', 'edulearn.edu'],
                    'industry': ['IT Services', 'Agriculture', 'Healthcare', 'Travel', 'Education'],
                    'location': ['San Francisco', 'London', 'Boston', 'New York', 'Chicago']
                }

                sample_df = pd.DataFrame(sample_data)
                st.session_state.original_df = sample_df

                # Display sample data
                st.subheader("Sample Data")
                st.dataframe(sample_df)

                # Show processing button
                if st.button("Process Sample Data"):
                    # Set up a progress bar
                    st.subheader("Processing Progress")
                    progress_bar = st.progress(0, text="Starting...")

                    # Process the data
                    with st.spinner('Searching the web for company information...'):
                        enriched_df = enrich_company_data(sample_df, progress_bar)

                    # Store enriched data in session state
                    st.session_state.enriched_df = enriched_df

                    # Success message
                    st.success(f"✅ Successfully processed {len(enriched_df)} companies!")

                    # Prompt to view results
                    st.info("Click on the 'Results & Visualization' tab to view the enriched data.")

    # Tab 2: Results and Visualization
    with tab2:
        st.header("Enriched Data Results")

        if 'enriched_df' in st.session_state:
            # Display enriched data
            st.subheader("Enriched Company Data")
            st.dataframe(st.session_state.enriched_df)

            # Create visualizations
            create_visualizations(st.session_state.enriched_df)
        else:
            st.info("No enriched data available yet. Please upload and process data in the first tab.")

    # Tab 3: Export
    with tab3:
        st.header("Export Enriched Data")

        if 'enriched_df' in st.session_state:
            # Add timestamp to filename
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"enriched_company_data_{timestamp}.csv"

            # Convert dataframe to CSV
            csv = st.session_state.enriched_df.to_csv(index=False)

            # Download button
            st.download_button(
                label="Download Enriched Data as CSV",
                data=csv,
                file_name=filename,
                mime="text/csv",
                key="download-csv"
            )

            # Show export to CRM options (for future implementation)
            st.subheader("Export to CRM (Coming Soon)")

            # Create columns for CRM options
            crm_col1, crm_col2, crm_col3 = st.columns(3)

            with crm_col1:
                st.button("Export to Salesforce", disabled=True)

            with crm_col2:
                st.button("Export to HubSpot", disabled=True)

            with crm_col3:
                st.button("Export to Custom API", disabled=True)

            st.info("CRM integration will be available in a future update.")

        else:
            st.info("No enriched data available yet. Please upload and process data in the first tab.")

# Run the app
if __name__ == "__main__":
    main()

Overwriting app.py


In [ ]:
!ngrok authtoken 2ib50v1FOA6wAqxOTCfZoo9FlMs_6z9Xpydw78W9yFkYy7jSN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
!streamlit run app.py &>/dev/null&
ngrok_tunnel = ngrok.connect(8501)
print(f"Streamlit is running at: {ngrok_tunnel.public_url}")

Streamlit is running at: https://f21d-35-194-253-178.ngrok-free.app
